In [1]:
import pandas as pd
meta = pd.read_csv("meta_drama.csv", encoding = "utf8")
meta.head()

,Unnamed: 0,id,title,author,period,type,genre,date,file,source,annotation,tokens_cleaned
0,0,1,Der sterbende Cato,Gottsched,Aufklaerung,Tragedy,drama,1731,gottsched-der-sterbende-cato.xml,https://github.com/dracor-org/gerdracor,NaN,22047
1,1,2,Ein Deutsches Vorspiel,Neuber,Aufklaerung,NaN,drama,1734,neuber-ein-deutsches-vorspiel.xml,https://github.com/dracor-org/gerdracor,NaN,6480
2,2,3,Die Pietisterey im Fischbein-Rocke oder Die Do...,Gottsched,Aufklaerung,Comedy,drama,1736,gottschedin-die-pietisterey-im-fischbein-rocke...,https://github.com/dracor-org/gerdracor,NaN,27691
3,3,4,Die von der Weisheit wider die Unwissenheit be...,Neuber,Aufklaerung,NaN,drama,1736,neuber-die-beschuetzte-schauspielkunst.xml,https://github.com/dracor-org/gerdracor,NaN,7377
4,4,5,Die Verehrung der Vollkommenheit durch die geb...,Neuber,Aufklaerung,NaN,drama,1737,neuber-die-verehrung-der-vollkommenheit.xml,https://github.com/dracor-org/gerdracor,NaN,7091


In [2]:
def load_corpus(path, df):
    from numpy import append 

    texts = []
    filenames = []
    for filename in df["file"]:
        with open(path + filename, 'r', encoding="utf16") as f:
            texts.append(f.read())
            f.close()
        filenames.append(filename)    
    return texts, filenames

In [3]:
def load_corpus_drama(path):
    from numpy import append 
    from os import listdir
    
    texts = []
    filenames = []
    for filename in listdir(path):
        with open(path + "/" + filename, 'r', encoding="utf8") as f:
            texts.append(f.read())

        filenames.append(filename)    
    return texts, filenames

In [4]:
# texts, filenames = load_corpus("corpora/cleaned_normalized/corpus_drama/", meta)
texts, filenames = load_corpus_drama("corpora/cleaned_normalized/corpus_drama/")

In [5]:
sent = [text.split() for text in texts]

In [6]:
from gensim.models.phrases import Phrases, Phraser
import gensim.models
import multiprocessing

In [7]:
def train_model(words):
    phrases = Phrases(words, min_count=30, progress_per=10000) # detects multi word expressions

    bigram = Phraser(phrases)
    sentences = bigram[words]
    
    model = gensim.models.FastText(min_count=10,
                                   vector_size = VECTOR_SIZE,
                                   window = WINDOW,
                                   alpha=ALPHA,
                                   workers = multiprocessing.cpu_count()-2,
                                   negative = NEGATIVE,
                                   seed = 42)

    model.build_vocab(corpus_iterable=sentences)

    model.train(corpus_iterable=sentences,
                        epochs=EPOCHS,
                        total_examples=model.corpus_count)
    
    return model

In [8]:
def model_evaluation(lang_model):
    odd_one_out =["adel volk republik monarchie gericht wohlstand", #staatstheorie wohlstand
                  "gedicht roman buch urheber druck roman gesetz", #schriftstellertum gesetz
                  "haus wohlstand reichtum handwerk familie könig", #bürgertum könig
                  "philosophie verstand gelassenheit ethik ruhe zorn", #vernunft zorn
                  "trauer wut freude glück zorn verstand", #gefühle verstand
                  "mutter königin geburt maid künstlerin krieger", #frauenbild krieger
                  "bild lied statur gedicht künstler acker", #kunst acker 
                  "ritter schild jäger maid jungfer pistole", #vergangeheit pistole
                  "dämon fee drache magie zauberer krankheit", #phantastisch krankheit
                  "gott heilig priester kloster pfaffe schmied", #religion schmied
                  "fluss gebirge sturm blume ozean mine", #natur mine
                  "angst monster geister friedhof wahnsinn sonne", #nacht sonne
                  "ausbildung reife erwachsen erkenntnis lebensweg kind" #bildung kind
                  ] 

    odd_one_out = [string.split() for string in odd_one_out]

    odd_one = ["wohlstand",
               "gesetz",
               "könig",
               "zorn",
               "verstand",
               "krieger",
               "acker",
               "pistole",
               "krankheit",
               "schmied",
               "mine",
               "sonne",
               "kind"]

    correct = 0
    for i in range(0,len(odd_one_out)):
        if lang_model.wv.doesnt_match(odd_one_out[i]) == odd_one[i]:
            correct += 1

    return correct/len(odd_one_out)

In [9]:
# 72 parameter

vector_size = [200, 300]
window = [5, 15, 20]
epochs = [30, 50]
negative = [5, 10, 20]
alpha = [0.01, 0.1]

In [10]:
f = open("finetuning.csv", "w")
f.write("vector_size,window,epoch,negative,alpha,score\n")
f.close()

In [11]:
i = 0
for VECTOR_SIZE in vector_size:
    for WINDOW in window:
        for EPOCHS in epochs:
            for NEGATIVE in negative:
                for ALPHA in alpha:
                    i += 1
                    if i < 1:
                        continue
                    else:    
                        print(i)
                        model = train_model(sent)
                        score = model_evaluation(model)
                        f = open("finetuning.csv", "a")
                        f.write(str(VECTOR_SIZE) + "," + str(WINDOW) + "," + str(EPOCHS) + "," + str(NEGATIVE) + "," + str(ALPHA) + "," + str(score) + "\n")
                        f.close()                  

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
